# descarga de lista de todas las mesas nacionales e internacionales, y guardar en json's

en esta seccion se descarga los jsons que nos servira para descargar todas las actas nacionales y extranjeras

In [1]:
# descargar datos de url y guardar el json
import requests
import json
import os

base_url = "https://computo.oep.org.bo/"


nacional_response = requests.get(base_url + "geografiaNacional.json")
internacional_response = requests.get(base_url + "geografiaExtranjero.json")

# descargar el archivo geografiaNacional.json
if nacional_response.status_code == 200:
    with open("geografiaNacional.json", "w", encoding="utf-8") as nacional_file:
        json.dump(nacional_response.json(), nacional_file, ensure_ascii=False, indent=2)
        print("Archivo geografiaNacional.json descargado y guardado correctamente.")
else:
    print(f"Error al descargar geografiaNacional.json: {nacional_response.status_code}")

# Guardar el archivo geografiaExtranjero.json
if internacional_response.status_code == 200:
    with open("geografiaExtranjera.json", "w", encoding="utf-8") as internacional_file:
        json.dump(internacional_response.json(), internacional_file, ensure_ascii=False, indent=2)
        print("Archivo geografiaExtranjero.json descargado y guardado correctamente.")
else:
    print(f"Error al descargar geografiaExtranjera.json: {internacional_response.status_code}")

Archivo geografiaNacional.json descargado y guardado correctamente.
Archivo geografiaExtranjero.json descargado y guardado correctamente.


descargamos la libreria tqdm para tener el proceso

In [2]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


aqui creamos la descarga de las actas nacionales en 8 hilos, 8 imagens de actas a la vez, depende de tu red el timepo que tardara

In [3]:
import json
import os
import requests
import base64
import time
import csv
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Configuración
JSON_FILE = "geografiaNacional.json"
BASE_DIR = "ACTAS_1000"  # Carpeta única para todas las actas
LOG_FILE = "descargas_1000.csv"
URL = "https://computo.oep.org.bo/api/v1/resultados/mesa"
MAX_THREADS = 8         # hilos paralelos
MIN_DELAY = 0.3         # pausa mínima entre descargas
MAX_DELAY = 0.7         # pausa máxima entre descargas
MAX_ACTAS = 1000        # Límite de 1000 actas

# Cargar JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear carpeta única
os.makedirs(BASE_DIR, exist_ok=True)

# Función para descargar una mesa individual con numeración
def descargar_mesa(mesa_id, mesa_num, numero_secuencial):
    mesa_file = os.path.join(BASE_DIR, f"acta_{numero_secuencial:04d}.jpg")
    
    if os.path.exists(mesa_file):
        return numero_secuencial, mesa_id, mesa_file, "Ya descargado"
    
    try:
        resp = requests.post(URL, json={"codigoMesa": mesa_id}, timeout=15)
        if resp.status_code == 200:
            data_resp = resp.json()
            estado = "Sin ACTA"
            for adj in data_resp.get("adjunto", []):
                if adj["tipo"] == "ACTA" and adj["valor"] != "false":
                    img_data = base64.b64decode(adj["valor"])
                    with open(mesa_file, "wb") as f:
                        f.write(img_data)
                    del img_data
                    estado = "OK"
                    break
            return numero_secuencial, mesa_id, mesa_file, estado
        else:
            return numero_secuencial, mesa_id, "", f"Error HTTP {resp.status_code}"
    except Exception as e:
        return numero_secuencial, mesa_id, "", f"Error {e}"

# Preparar lista de tareas (limitado a 1000)
tareas = []
contador = 0

for pais in data:
    if contador >= MAX_ACTAS:
        break
    for dep in pais.get("d", []):
        if contador >= MAX_ACTAS:
            break
        for prov in dep.get("p", []):
            if contador >= MAX_ACTAS:
                break
            for mun in prov.get("m", []):
                if contador >= MAX_ACTAS:
                    break
                for loc in mun.get("l", []):
                    if contador >= MAX_ACTAS:
                        break
                    for rec in loc.get("r", []):
                        if contador >= MAX_ACTAS:
                            break
                        for mesa in rec.get("t", []):
                            if contador >= MAX_ACTAS:
                                break
                            contador += 1
                            tareas.append((mesa["i"], mesa["n"], contador))

print(f"Se procesarán {len(tareas)} mesas (máximo {MAX_ACTAS})")

# Ejecutar descargas paralelas
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor, \
     open(LOG_FILE, "w", newline="", encoding="utf-8") as log:

    writer = csv.writer(log)
    writer.writerow(["Numero_Secuencial", "Mesa_ID", "Ruta", "Estado"])

    futures = {executor.submit(descargar_mesa, tid, tnum, tseq): (tseq, tid) for tid, tnum, tseq in tareas}

    with tqdm(total=len(tareas), desc="Descargando actas") as pbar:
        for fut in as_completed(futures):
            numero_secuencial, mesa_id, mesa_file, estado = fut.result()
            writer.writerow([numero_secuencial, mesa_id, mesa_file, estado])
            # Pausa aleatoria pequeña para proteger servidor
            time.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
            pbar.update(1)

print("Descarga finalizada ✅")
print(f"Las actas se encuentran en la carpeta: {BASE_DIR}")
print(f"Archivos numerados desde acta_0001.jpg hasta acta_{len(tareas):04d}.jpg")

Se procesarán 1000 mesas (máximo 1000)


Descargando actas: 100%|██████████| 1000/1000 [08:23<00:00,  1.99it/s]

Descarga finalizada ✅
Las actas se encuentran en la carpeta: ACTAS_1000
Archivos numerados desde acta_0001.jpg hasta acta_1000.jpg


actas internacionales, igual en 8 hilos concurrentes

In [4]:
import json
import os
import requests
import base64
import time
import csv
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Configuración
JSON_FILE = "geografiaExtranjera.json"
BASE_DIR = "ACTAS_EXTRANJERAS_1000"  # Carpeta única para todas las actas extranjeras
LOG_FILE = "descargas_extranjeras_1000.csv"
URL = "https://computo.oep.org.bo/api/v1/resultados/mesa"
MAX_THREADS = 8         # hilos paralelos
MIN_DELAY = 0.3         # pausa mínima entre descargas
MAX_DELAY = 0.7         # pausa máxima entre descargas
MAX_ACTAS = 1000        # Límite de 1000 actas

# Cargar JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear carpeta única
os.makedirs(BASE_DIR, exist_ok=True)

# Función para descargar una mesa individual con numeración
def descargar_mesa(mesa_id, mesa_num, numero_secuencial):
    mesa_file = os.path.join(BASE_DIR, f"acta_extranjera_{numero_secuencial:04d}.jpg")
    
    if os.path.exists(mesa_file):
        return numero_secuencial, mesa_id, mesa_file, "Ya descargado"
    
    try:
        resp = requests.post(URL, json={"codigoMesa": mesa_id}, timeout=15)
        if resp.status_code == 200:
            data_resp = resp.json()
            estado = "Sin ACTA"
            for adj in data_resp.get("adjunto", []):
                if adj["tipo"] == "ACTA" and adj["valor"] != "false":
                    img_data = base64.b64decode(adj["valor"])
                    with open(mesa_file, "wb") as f:
                        f.write(img_data)
                    del img_data
                    estado = "OK"
                    break
            return numero_secuencial, mesa_id, mesa_file, estado
        else:
            return numero_secuencial, mesa_id, "", f"Error HTTP {resp.status_code}"
    except Exception as e:
        return numero_secuencial, mesa_id, "", f"Error {e}"

# Preparar lista de tareas (limitado a 1000)
tareas = []
contador = 0

for pais in data:
    if contador >= MAX_ACTAS:
        break
    for ciudad in pais.get("c", []):
        if contador >= MAX_ACTAS:
            break
        for recinto in ciudad.get("r", []):
            if contador >= MAX_ACTAS:
                break
            for mesa in recinto.get("t", []):
                if contador >= MAX_ACTAS:
                    break
                contador += 1
                tareas.append((mesa["i"], mesa["n"], contador))

print(f"Se procesarán {len(tareas)} mesas extranjeras (máximo {MAX_ACTAS})")

# Ejecutar descargas paralelas
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor, \
     open(LOG_FILE, "w", newline="", encoding="utf-8") as log:

    writer = csv.writer(log)
    writer.writerow(["Numero_Secuencial", "Mesa_ID", "Ruta", "Estado"])

    futures = {executor.submit(descargar_mesa, tid, tnum, tseq): (tseq, tid) for tid, tnum, tseq in tareas}

    with tqdm(total=len(tareas), desc="Descargando actas extranjeras") as pbar:
        for fut in as_completed(futures):
            numero_secuencial, mesa_id, mesa_file, estado = fut.result()
            writer.writerow([numero_secuencial, mesa_id, mesa_file, estado])
            # Pausa aleatoria pequeña para proteger servidor
            time.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
            pbar.update(1)

print("Descarga finalizada ✅")
print(f"Las actas extranjeras se encuentran en la carpeta: {BASE_DIR}")
print(f"Archivos numerados desde acta_extranjera_0001.jpg hasta acta_extranjera_{len(tareas):04d}.jpg")

Se procesarán 1000 mesas extranjeras (máximo 1000)


Descargando actas extranjeras: 100%|██████████| 1000/1000 [08:23<00:00,  1.99it/s]

Descarga finalizada ✅
Las actas extranjeras se encuentran en la carpeta: ACTAS_EXTRANJERAS_1000
Archivos numerados desde acta_extranjera_0001.jpg hasta acta_extranjera_1000.jpg
